In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import product
import pandas as pd
import seaborn as sns

pq0, alfa = 0.01 , 0.4

In [106]:
def constroe_estados(profundidade,frota):
    aux = []
    for i in range(profundidade+1):
        for j in range(i+1):
            aux.append((i,j))
    
    estados = list(product(*[aux for i in range(frota)]))
    estados.append(tuple([(-1,-1) for i in range(frota)]))
    acoes = list(product(*[[0,1,2] for i in range(frota)]))
    return estados,acoes

def proximos(estado,acao,prof):
    frota = len(acao)
    aux = [[]]*frota
    pr_ax = [[]]*frota
    rc_ax = [[]]*frota
    cnd = [[]]*frota
    for j,v in enumerate(acao):
        x,y = estado[j][0]+1,estado[j][1]+1
        rec = recompensa([x,y])
        cnd[j] = int(estado[j][0]>=prof)
        if v == 0:
            aux[j] = [(x,y),(x,0)]
            aux_q = pq0/(pq0+(1-pq0)*np.exp(-alfa*estado[j][1]))
            pr_ax[j] = [1-aux_q,aux_q]
            rc_ax[j] = [rec[0],rec[1]]
        elif v == 1:
            aux[j] = [(x,0)]  
            pr_ax[j] = [1]
            rc_ax[j] = [rec[2]]
        elif v == 2:
            aux[j] = [(0,0)]
            pr_ax[j] = [1]
            rc_ax[j] = [rec[3]]

    nest = list(product(*aux))
    probs = list(product(*pr_ax))
    recs = list(product(*rc_ax))
    probs = [np.prod(pr) for pr in probs]
    recs = [np.sum(rc) for rc in recs]
    
    if sum(cnd):
        nest = [tuple([(-1,-1) for i in range(frota)])]
        probs = [1]
        recs = [0]

    if estado[0][1]==-1:
        nest = [tuple([(-1,-1) for i in range(frota)])]
        probs = [1]
        recs = [0]

    return {"estados":nest,"probs":probs,"recs":recs}

def recompensa(estado):
    tvida = estado[0]

    valor_equipamento = np.exp(-0.005*tvida)
    valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)
    valor_defeito = -2*valor_manutencao/valor_equipamento
        
    res_a0_op,res_a0_de = 0.01/valor_equipamento,valor_defeito

    res_a1 = -valor_manutencao/valor_equipamento
    
    res_a2 = -(1 - 0.8*valor_equipamento)
    
    res = [res_a0_op,res_a0_de,res_a1,res_a2]
    if estado[0]==-1:
        res = [0,0,0,0]
    return res


def cria_politica(lista_estados):
    frota = len(lista_estados[0])
    politica = {}
    for est in lista_estados:
        aux = [0]*(3**frota)
        aux[0] = 1
        politica[est] = aux
    return politica

def avalia_politica(sims, lista_estados,acoes,politica):
    tol = 1e-5
    frota = len(lista_estados[0])
    prof = max([est[0][0] for est in lista_estados])

    VN = {v:1-2*random.random() for v in lista_estados}
    VN[tuple([(-1,-1) for i in range(frota)])] = 0
    
    cnt, nor = 0, 10
    
    M = {}
    M[0] = np.array(list(VN.values()))
    val_error = []
    while nor>tol and cnt <= sims:
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            pols = politica[est]
            v_aux = 0
            for aci,acv in enumerate(acoes):
                next = proximos(est,acv,prof)
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                for j_e in range(len(nest)):  
                    v_aux = v_aux + pols[aci]*probs[j_e]*(recs[j_e]+V[nest[j_e]])
           
            VN[est] = v_aux
        
        cnt = cnt+1
        M[cnt%2] = np.array(list(VN.values()))

        # nor = np.linalg.norm(M[cnt]-M[cnt-1])
        nor = np.linalg.norm(M[1]-M[0])
        val_error.append(nor)
    return M

def aprimora_politica(lista_estados,politica,sims = 50):
    # escolhe a melhor política deterministica
    tol = 1e-5
    prof = lista_estados[-2][0]
    
    VN = {v:1-2*random.random() for v in lista_estados}
    VN[(-1,-1)]=0
    
    # politica = {est:0 for v in lista_estados}

    P,M = {},{}
    PLN = {k:v for k,v in politica.items()}

    P[0] = politica
    
    cnt_pol = 0

    estavel = False

    while not estavel and cnt_pol<500:
        cnt_pol = cnt_pol+1
        
        PL = {k:v for k,v in PLN.items()}
        PLN = {}
        
        cnt, nor = 0, 10
        while nor>tol and cnt <= sims:
            
            V = {k:v for k,v in VN.items()}
            VN = {}

            for est in lista_estados:
                next = proximos(est,prof) 
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                pols = politica[est]

                v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
                v_a1 = probs[2]*(recs[2]+V[nest[2]])
                v_a2 = probs[3]*(recs[3]+V[nest[3]])

                VN[est] = pols[0]*v_a0+pols[1]*v_a1+pols[2]*v_a2
            
            cnt = cnt+1
            nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
        
        V = {k:v for k,v in VN.items()}
        M[cnt_pol-1] = {k:v for k,v in VN.items()}
        mud = 0
        for est in lista_estados:
            next = proximos(est,prof) 
            nest = next["estados"]
            probs = next["probs"]
            recs = next["recs"]
            pols = politica[est]
            v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
            v_a1 = probs[2]*(recs[2]+V[nest[2]])
            v_a2 = probs[3]*(recs[3]+V[nest[3]])
            idx_pol_max = np.argmax([v_a0,v_a1,v_a2])
            aux_pol = [0,0,0]
            aux_pol[idx_pol_max]=1
            PLN[est] = aux_pol
            
            if not pols[idx_pol_max] == 1:
                mud = mud+1

        if mud > 0:
            estavel = False
        else:
            estavel = True
            
        politica = {k:v for k,v in PLN.items()}
        P[cnt_pol] = {k:v for k,v in PLN.items()}

    cnt, nor = 0, 10
    while nor>tol and cnt <= sims:
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            next = proximos(est,prof) 
            nest = next["estados"]
            probs = next["probs"]
            recs = next["recs"]
            pols = politica[est]
            
            v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
            v_a1 = probs[2]*(recs[2]+V[nest[2]])
            v_a2 = probs[3]*(recs[3]+V[nest[3]])

            VN[est] = pols[0]*v_a0+pols[1]*v_a1+pols[2]*v_a2

        cnt = cnt+1
        nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
    
    M[cnt_pol] = {k:v for k,v in VN.items()}
    
    return P,M


def value_iteration(lista_estados,acoes,politica,sims = 500):
    # escolhe a melhor política deterministica

    tol = 1e-5
    frota = len(lista_estados[0])
    prof = max([est[0][0] for est in lista_estados])

    VN = {v:0.05 for v in lista_estados}
    VN[tuple([(-1,-1) for i in range(frota)])] = 0
    

    PLN = {k:np.argmax(v) for k,v in politica.items()}
    
    M,P = {},{}
    M[0] = np.array(list(VN.values()))
    P[0] = {k:v for k,v in PLN.items()}
    
    cnt, nor = 0, 10
    val_norm = []
    while nor>tol and cnt <= sims:
        
        PL = {k:v for k,v in PLN.items()}
        PLN = {}
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            # pols = politica[est]
            v_aux = []
            for aci,acv in enumerate(acoes):
                next = proximos(est,acv,prof)
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                v_aux2 = 0
                for j_e in range(len(nest)):  
                    v_aux2 = v_aux2 + probs[j_e]*(recs[j_e]+V[nest[j_e]])
                v_aux.append(v_aux2)

            VN[est] = np.max(v_aux)
            PLN[est] = np.argmax(v_aux)
            
        cnt = cnt+1
        nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
        val_norm.append(nor)
        print(f"sim: {cnt},norma: {nor}",end='\r')
        M[cnt%2] = np.array(list(VN.values()))
        P[cnt%2] = {k:v for k,v in PLN.items()}
    
    return {"politica":P, "valor":M, "norma":val_norm}

def gera_episodio(T,acoes,politica,est0):
    prof = max([v[0][0] for v in politica.keys()])
    
    if not isinstance(politica[est0],list):
        frota = len(est0)
        len_ac = 3**frota
        for esta in politica.keys():
            aux = [0]*len_ac
            aux[politica[esta]] = 1
            politica[esta] =  aux
        
    est = tuple([x for x in est0])
    t = 0
    ES, AC, RC =[], [], []
    
    while t<T and est[0][0]>-1:
        t = t+1
        polis = np.cumsum(politica[est])
        aux_ac = random.random()
        ac = np.sum([int(x<aux_ac) for x in polis])
        nxt = proximos(est,acoes[ac],prof)
        ee = random.random()
        aux_ee = np.sum([int(x<ee) for x in np.cumsum(nxt["probs"])])
        nest = nxt["estados"][aux_ee]
        rec = nxt["recs"][aux_ee]
        ES.append(est)
        AC.append(acoes[ac])
        RC.append(rec)
        est = tuple([x for x in nest])

    
    return {'estados':ES,'acoes':AC,'recs':RC}

def FirstVisitMC(T,est0,prof,frota,n_eps = 1000,pols=[]):
    
    lista_estados, lista_acoes = constroe_estados(prof,frota)
    
    # pols = cria_politica(lista_estados)
    # pols = {}
    # for est in lista_estados:
    #     pols[est] = [0.9,0.1,0.0]

    if pols==[]:        
        pols = {}
        for est in lista_estados:
            aux = [0]*(3**frota)
            aux[0] = 1
            pols[est] = aux
    
    N = {v:0 for v in lista_estados}
    R = {v:0 for v in lista_estados}
    V = {v:0 for v in lista_estados}
    
    for n in range(n_eps):
        epi = gera_episodio(T,lista_acoes,pols,est0)

        estados = [epi['estados'][t] for t in range(len(epi["estados"])-1,-1,-1)]
        
        G = 0
        for t in range(len(epi["estados"])-2,-1,-1):
            G = G + epi["recs"][t]
            est = epi["estados"][t]
            if est not in estados[:t]:
                R[est] = R[est] + G
                N[est] = N[est] + 1
                # V[est] = V[est]+(G-V[est])/N[est]
                V[est] = V[est]+0.1*(G-V[est])

    # for est in lista_estados:
    #     V[est] = R[est]/(N[est]+1e-15)               

    return {"valor":V,"nvezes":N,"retorno":R}

def OnPolicyFirstVisitMC(T,profundidade,est0=(0,0),n_eps = 1000,tol=0.15):
    # escolhe a melhor política deterministica
    _, lista_estados = constroe_estados(profundidade)
    
    pols = {}
    lista_estados_acoes = []
    for est in lista_estados:
        aux = [0.8,.2,.0]
        pols[est] = [x/sum(aux) for x in aux]
        for i in range(3):
            lista_estados_acoes.append(((est[0],est[1]),i))


    
    prof = profundidade
    N = {v:1e-15 for v in lista_estados_acoes}
    R = {v:0 for v in lista_estados_acoes}
    V = {v:0 for v in lista_estados_acoes}
    for n in range(n_eps):
        epi = gera_episodio(prof,T,pols,random.choice(lista_estados))

        estados_acoes = [(epi[x][0],epi[x][1]) for x in range(len(epi)-1,-1,-1)]
        estados_acoes = [(epi[x][0],epi[x][1]) for x in range(len(epi))]
        
        G = 0
        for t in range(len(epi)-2,-1,-1):
            G = G + epi[t][2]
            est = (epi[t][0],epi[t][1])
            if est not in estados_acoes[:t]:
                N[est] = N[est] + 1
                V[est] = V[est]+(G-V[est])/N[est]
                vals = [-100,-100,-100]
                for i in range(3):
                    if N[(est[0],i)]>=1:
                        vals[i] = V[(est[0],i)]
                acm = np.argmax(vals)
                aux_pol = [tol/3,tol/3,tol/3]
                aux_pol[acm] = 1-tol+tol/3
                pols[est[0]] = aux_pol

    return N,R,V,pols

In [ ]:
prof = 27
est_list,acoes_list = constroe_estados(prof,2)
polis = cria_politica(est_list)
# M = avalia_politica(10,est_list,acoes_list,polis)
R = value_iteration(est_list,acoes_list,polis)
# x = [a[0] for a,b in zip(P[0].items(),P[].items()) if not a[1]==b[1]]

In [75]:
R["valor"]

{0: array([0.36454088, 0.35285845, 0.34905536, ..., 0.        , 0.        ,
        0.        ]),
 1: array([0.36454088, 0.35285845, 0.34905536, ..., 0.        , 0.        ,
        0.        ])}

In [66]:
epi = gera_episodio(20,acoes_list,P1,((0,0),(0,0)))
epi["estados"]

[((0, 0), (0, 0)),
 ((1, 1), (1, 1)),
 ((2, 2), (2, 2)),
 ((3, 3), (3, 3)),
 ((4, 4), (4, 4)),
 ((5, 0), (5, 5)),
 ((6, 1), (6, 6)),
 ((7, 2), (7, 7)),
 ((8, 3), (8, 0)),
 ((9, 4), (9, 1)),
 ((10, 5), (10, 2)),
 ((11, 6), (11, 3)),
 ((12, 7), (12, 0)),
 ((13, 8), (13, 1)),
 ((14, 0), (14, 2)),
 ((15, 1), (15, 3)),
 ((16, 2), (16, 4)),
 ((17, 3), (17, 5)),
 ((18, 4), (18, 6)),
 ((19, 5), (19, 0))]

In [94]:
len(epi["estados"])

20

In [107]:
RS = FirstVisitMC(prof,((0,0),(0,0)),prof,2,n_eps = 10000,pols = P1)

In [108]:
{k:v for k,v in RS["valor"].items() if v>0}

{((0, 0), (0, 0)): 0.3576402725483779,
 ((1, 0), (1, 0)): 0.032502337752019564,
 ((1, 0), (1, 1)): 0.3425622043968622,
 ((1, 1), (1, 0)): 0.34079990597230625,
 ((1, 1), (1, 1)): 0.33721762711745545,
 ((2, 0), (2, 0)): 0.06347784696723702,
 ((2, 0), (2, 1)): 0.05171961236808603,
 ((2, 0), (2, 2)): 0.3326945271145036,
 ((2, 1), (2, 1)): 0.030482237417851228,
 ((2, 1), (2, 2)): 0.32236148620883287,
 ((2, 2), (2, 0)): 0.3399042300968352,
 ((2, 2), (2, 1)): 0.32067383680899597,
 ((2, 2), (2, 2)): 0.31706169102538123,
 ((3, 0), (3, 0)): 0.03748632830888592,
 ((3, 0), (3, 1)): 0.10084876247094056,
 ((3, 0), (3, 2)): 0.07131986227647781,
 ((3, 0), (3, 3)): 0.3025317293904865,
 ((3, 1), (3, 1)): 0.05962041732169728,
 ((3, 1), (3, 2)): 0.0478621827225463,
 ((3, 1), (3, 3)): 0.31238996642084255,
 ((3, 2), (3, 0)): 0.05615747716346787,
 ((3, 2), (3, 2)): 0.02845201128861979,
 ((3, 2), (3, 3)): 0.30202493926248136,
 ((3, 3), (3, 0)): 0.2955049512608996,
 ((3, 3), (3, 1)): 0.3191710436059674,
 ((3, 

In [ ]:
N,R,V,P= OnPolicyFirstVisitMC(20,profundidade = 20,n_eps=100000)


In [ ]:
xx = {k:np.argmax(v) for k,v in P.items()}
[k for k,v in xx.items() if v>0]

In [ ]:
x = [a[0] for a,b in zip(P[0].items(),P[112].items()) if not a[1]==b[1]]
y = [k for k,v in P[112].items() if v==1]
y
